### part 1 - Tokenization

In [1]:
# import the library
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [ ]:
# define the sentences to be tokenize
sentences = [
    'I  love my dog',
    'I love my cat'
]

#define the tokenizer, the num_word is the maximum num of word to keep
tokenizer = Tokenizer(num_words=100)
#go through all the text and tokeniz it
tokenizer.fit_on_texts(sentences)
#conaining the full list of tokenized words 
word_index = tokenizer.word_index
print(word_index)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'cat': 5}


### part 2 - Sequencing - Turning sentences into data

In [ ]:
# define the sentences to be tokenize
sentences = [
    'I  love my dog',
    'I love my cat',
    'You love my dog!',
    'Do you think my dog is amazing?'
]

#define the tokenizer, the num_word is the maximum num of word to keep, 
#OOV used to create token and use it to replace words that it does not recognize
tokenizer = Tokenizer(num_words=100,  oov_token="<OOV>")
#go through all the text and tokeniz it
tokenizer.fit_on_texts(sentences)
#conaining the full list of tokenized words 
word_index = tokenizer.word_index
#creates sequences of tokens represinting each sentence
sequences = tokenizer.texts_to_sequences(sentences)

print("word tokenization: ", word_index)
print("sequences of tokens: ", sequences)

# Try with words that the tokenizer wasn't fit to
test_data = [
    'i really love my dog',
    'my dog loves my manatee'
]
#creates sequences of tokens represinting each sentence
test_seq = tokenizer.texts_to_sequences(test_data)
print("Test: ", test_seq)
#add padding to sentences to get same length for each sentence
#maxlen is the maximum length for each sentence, if not it will pad sentences based on the longest sentence
#use padding='post' to padd the sentences from the right
#use truncation = 'post' so if the sentences are longer than the specified length
padded = pad_sequences(test_seq, maxlen=7)
print("padded test: ", padded)

word tokenization:  {'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}
sequences of tokens:  [[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]]
Test:  [[5, 1, 3, 2, 4], [2, 4, 1, 2, 1]]
padded test:  [[0 0 5 1 3 2 4]
 [0 0 2 4 1 2 1]]


### part 3 - Training a model to recognize sentiment in text

install the corpus

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json

--2023-06-02 02:35:53--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 172.217.204.128, 74.125.134.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2023-06-02 02:35:53 (173 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
#Set the parameters
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

#load the installed corpus 
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)


#loop into the corpos to append the sentences and labels
sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])


#divide the data to training and testig
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]


#define the tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
#create token for every word in the corpus
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
#creates sequences of tokens for the training set
training_sequences = tokenizer.texts_to_sequences(training_sentences)
#pad the training sequnces
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
#creates sequences of tokens for the testing set
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
#pad the testing sequnces
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)



#convert the training and testing sets to numpay array
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)


#define the model hyper-parameter
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



#train the model
num_epochs = 3
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/3
625/625 - 14s - loss: 0.6733 - accuracy: 0.5730 - val_loss: 0.6242 - val_accuracy: 0.5825 - 14s/epoch - 23ms/step
Epoch 2/3
625/625 - 5s - loss: 0.4706 - accuracy: 0.8094 - val_loss: 0.4009 - val_accuracy: 0.8323 - 5s/epoch - 8ms/step
Epoch 3/3
625/625 - 3s - loss: 0.3295 - accuracy: 0.8670 - val_loss: 0.3601 - val_accuracy: 0.8496 - 3s/epoch - 4ms/step


In [ ]:
#test sentences
sentence = ["granny starting to fear spiders in the garden might be real", "game of thrones season finale showing this sunday night"]
#creates sequences of tokens for the test sentences
sequences = tokenizer.texts_to_sequences(sentence)
#pad the test sequnces
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
#predict the label for test sequnces
print(model.predict(padded))

1/1 [==============================] - 0s 111ms/step
[[0.24872854]
 [0.35498986]]


### part 5 - Long Short-Term Memory for NLP

In [ ]:
#define LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.LSTM(64)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
#define stacked LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    # return_sequences=True used to feed the next layer lstm
    tf.keras.layers.Bidirectional(tf.keras.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

### part 6 - Training an AI to create poetry

in this part i'm going to use KSUCCA corpus 

In [2]:
#define the tokenizer
tokenizer = Tokenizer()
#read the corpus
data = open('/content/sample_data/aa1.txt').read()
#split the corpus , it will convert it to list
corpus = data.split("\n")
#tokenize the corpus
tokenizer.fit_on_texts(corpus)
#get the number of word in the corpus
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)


{'من': 1, 'الله': 2, 'في': 3, 'ما': 4, 'إن': 5, 'الذين': 6, 'لا': 7, 'على': 8, 'ولا': 9, 'إلا': 10, 'وما': 11, 'أن': 12, 'قال': 13, 'إلى': 14, 'لهم': 15, 'يا': 16, 'ومن': 17, 'لكم': 18, 'به': 19, 'ثم': 20, 'كان': 21, 'بما': 22, 'قل': 23, 'الأرض': 24, 'أو': 25, 'ذلك': 26, 'له': 27, 'آمنوا': 28, 'وإن': 29, 'هم': 30, 'قالوا': 31, 'هو': 32, 'الذي': 33, 'كل': 34, 'والله': 35, 'كانوا': 36, 'فيها': 37, 'عن': 38, 'عليهم': 39, 'ربك': 40, 'يوم': 41, 'إذا': 42, 'كنتم': 43, 'السماوات': 44, 'شيء': 45, 'هذا': 46, 'كفروا': 47, 'الناس': 48, 'لم': 49, 'وهو': 50, 'فإن': 51, 'الكتاب': 52, 'إذ': 53, 'عليكم': 54, 'والذين': 55, 'والأرض': 56, 'منهم': 57, 'فلا': 58, 'عذاب': 59, 'بعد': 60, 'أيها': 61, 'عليه': 62, 'إنا': 63, 'حتى': 64, 'بالله': 65, 'وهم': 66, 'إنه': 67, 'موسى': 68, 'أولئك': 69, 'ولقد': 70, 'قوم': 71, 'إني': 72, 'فيه': 73, 'أم': 74, 'رب': 75, 'قبل': 76, 'قد': 77, 'عند': 78, 'بل': 79, 'لله': 80, 'الدنيا': 81, 'ولكن': 82, 'خير': 83, 'يشاء': 84, 'ولو': 85, 'الحق': 86, 'وإذا': 87, 'مما': 88, 'إنما':

In [3]:
input_sequences = []
#go throug the corpus
for line in corpus:
  #create list of token for each line in the corpus
	token_list = tokenizer.texts_to_sequences([line])[0]
  #go through the current list of token and generate n-gram for it
  #for example, the current list of token = [1 2 3 4] it will be [1 2] [1 2 3] [1 2 3 4]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

#get the max length in the n-gram list
max_sequence_len = max([len(x) for x in input_sequences])
# pad sequences and convert it to numpy array
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
#one-hoe encoding for the label
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [4]:
#define LSTM model and hyper-parameter
model = Sequential()
                                                  # -1 for the label
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=10, verbose=1)
#print model.summary()
print(model)


Epoch 1/10
2100/2100 [==============================] - 86s 36ms/step - loss: 7.8767 - accuracy: 0.0468
Epoch 2/10
2100/2100 [==============================] - 38s 18ms/step - loss: 6.9080 - accuracy: 0.0848
Epoch 3/10
2100/2100 [==============================] - 38s 18ms/step - loss: 6.0221 - accuracy: 0.1289
Epoch 4/10
2100/2100 [==============================] - 35s 17ms/step - loss: 5.1882 - accuracy: 0.1771
Epoch 5/10
2100/2100 [==============================] - 35s 17ms/step - loss: 4.4352 - accuracy: 0.2375
Epoch 6/10
2100/2100 [==============================] - 35s 16ms/step - loss: 3.7685 - accuracy: 0.3116
Epoch 7/10
2100/2100 [==============================] - 35s 17ms/step - loss: 3.1769 - accuracy: 0.4025
Epoch 8/10
2100/2100 [==============================] - 34s 16ms/step - loss: 2.6631 - accuracy: 0.4875
Epoch 9/10
2100/2100 [==============================] - 35s 17ms/step - loss: 2.2372 - accuracy: 0.5635
Epoch 10/10
2100/2100 [==============================] - 34s 16m

### test sample

In [10]:
seed_text = "وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن"
next_words = 10
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 21ms/step
وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء قالوا إنما أنت بشر ولئن اتبعت أهواءهم من بعد
